In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    
    if PARAM['atmpro'] == PARAM['atmpro']:
        atmpro = PARAM['atmpro']
    else:
        atmpro = 'mls'
        
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'n2o': 3.2e-07}
band = [3]
atmpro = trp
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o n2o band3a
  atmpro = mls
  band = 3a
  commitnumber = a06b618
  conc = 3.2e-07
  dv = 0.001
  klin = 2.22e-20
  molecule = n2o
  ng_adju = [0, 0]
  ng_refs = [1, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 620
  vmin = 540
  w_diffuse = [(1.8,), (1.66, 1.8)]
  wgt = [(0.9,), (0.5, 0.95)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.001855                    1  0.006638
0.000750        2  0.002047                    2  0.006790
0.001052        3  0.002444                    3  0.007119
0.001476        4  0.002849                    4  0.007446
0.002070        5  0.003258                    5  0.007778
0.002904        6  0.003668                    6  0.008109
0.004074        7  0.004080                    7  0.008447
0.005714        8  0.004494                    8  0.008786
0.008015        9  0.004907                    9  0.009125
0.011243       10  0.005322                   10  0.009469
0.015771       11  0.005715                   11  0.009811
0.022122       12  0.006092                   12  0.010156
0.031031       13  0.006444                   13  0.010502
0.043528       14  0.006890                   14  0.010950
0.061057       15  0.008756                   15  0.012567
0.085645       16  0.012771                   16  0.015943
0.120136       17  0.017812                   17  0.020193
0.168516       18  0.023229                   18  0.024801
0.236378       19  0.029012                   19  0.029762
0.331549       20  0.035167                   20  0.035079
0.465100       21  0.041694                   21  0.040768
0.652400       22  0.047892                   22  0.046217
0.915100       23  0.049528                   23  0.048041
1.283650       24  0.046452                   24  0.046056
1.800600       25  0.041716                   25  0.042576
2.525700       26  0.035857                   26  0.037898
3.542800       27  0.030188                   27  0.032963
4.969550       28  0.025118                   28  0.028084
6.970850       29  0.020616                   29  0.023204
9.778100       30  0.016569                   30  0.018330
13.715850      31  0.012803                   31  0.013428
19.239350      32  0.009138                   32  0.008565
26.987250      33  0.005425                   33  0.003814
37.855300      34  0.001487                   34 -0.000786
53.100050      35 -0.003774                   35 -0.005923
73.887500      36 -0.010115                   36 -0.011297
97.662500      37 -0.014579                   37 -0.014763
121.437500     38 -0.014752                   38 -0.014770
145.212500     39 -0.012417                   39 -0.012776
168.987500     40 -0.010324                   40 -0.010835
192.762500     41 -0.008392                   41 -0.008957
216.537500     42 -0.006572                   42 -0.007136
240.312500     43 -0.004832                   43 -0.005363
264.087500     44 -0.003163                   44 -0.003635
287.862500     45 -0.001553                   45 -0.001950
311.637500     46  0.000006                   46 -0.000304
335.412500     47  0.001520                   47  0.001301
359.187500     48  0.002994                   48  0.002871
382.962500     49  0.004431                   49  0.004407
406.737500     50  0.005836                   50  0.005910
430.512500     51  0.007211                   51  0.007381
454.287500     52  0.008560                   52  0.008822
478.062500     53  0.009885                   53  0.010239
501.837500     54  0.011189                   54  0.011630
525.612500     55  0.012473                   55  0.012991
549.387500     56  0.013741                   56  0.014328
573.162500     57  0.014995                   57  0.015646
596.937500     58  0.016235                   58  0.016943
620.712500     59  0.017458                   59  0.018220
644.487500     60  0.018660                   60  0.019464
668.262500     61  0.019816                   61  0.020658
692.037500     62  0.020873                   62  0.021760
715.812500     63  0.021757                   63  0.022715
739.587500     64  0.022452                   64  0.023514
763.362500     65  0.023035                   65  0.024214
78

# Fluxes by Layer

CRD                          CLIRAD                \
                        flug      fldg      fnetg       flug          fldg   
pressure    level                                                            
0.000000    1     -37.085128  0.000000 -37.085128 -37.140506  4.382719e-08   
0.000624    2     -37.085130  0.000001 -37.085128 -37.140507  1.149295e-06   
0.000876    3     -37.085130  0.000002 -37.085128 -37.140507  1.597217e-06   
0.001229    4     -37.085131  0.000003 -37.085128 -37.140507  2.232489e-06   
0.001723    5     -37.085132  0.000004 -37.085128 -37.140508  3.133433e-06   
0.002417    6     -37.085133  0.000005 -37.085128 -37.140509  4.411014e-06   
0.003391    7     -37.085135  0.000007 -37.085127 -37.140509  6.222704e-06   
0.004757    8     -37.085137  0.000011 -37.085127 -37.140511  8.792130e-06   
0.006672    9     -37.085140  0.000015 -37.085126 -37.140512  1.243580e-05   
0.009359    10    -37.085145  0.000021 -37.085124 -37.140515  1.760254e-05   
0.013128    11    -37.085152  0.000030 -37.085122 -37.140518  2.492933e-05   
0.018415    12    -37.085161  0.000043 -37.085118 -37.140522  3.531698e-05   
0.025830    13    -37.085173  0.000060 -37.085113 -37.140528  5.004434e-05   
0.036232    14    -37.085190  0.000086 -37.085105 -37.140536  7.092281e-05   
0.050823    15    -37.085214  0.000121 -37.085093 -37.140546  1.006124e-04   
0.071291    16    -37.085245  0.000174 -37.085072 -37.140560  1.442924e-04   
0.100000    17    -37.085282  0.000254 -37.085028 -37.140573  2.115028e-04   
0.140271    18    -37.085321  0.000378 -37.084943 -37.140581  3.162581e-04   
0.196760    19    -37.085358  0.000571 -37.084788 -37.140578  4.791269e-04   
0.275997    20    -37.085382  0.000867 -37.084515 -37.140551  7.315909e-04   
0.387100    21    -37.085373  0.001320 -37.084053 -37.140479  1.121541e-03   
0.543100    22    -37.085293  0.002011 -37.083282 -37.140327  1.722974e-03   
0.761700    23    -37.085089  0.003047 -37.082042 -37.140045  2.637864e-03   
1.068500    24    -37.084747  0.004505 -37.080242 -37.139616  3.954845e-03   
1.498800    25    -37.084299  0.006425 -37.077874 -37.139061  5.747602e-03   
2.102400    26    -37.083770  0.008879 -37.074892 -37.138399  8.130056e-03   
2.949000    27    -37.083223  0.011927 -37.071296 -37.137689  1.122140e-02   
4.136600    28    -37.082734  0.015685 -37.067049 -37.137015  1.518489e-02   
5.802500    29    -37.082419  0.020327 -37.062092 -37.136510  2.022233e-02   
8.139200    30    -37.082487  0.026101 -37.056386 -37.136417  2.655343e-02   
11.417000   31    -37.083301  0.033349 -37.049952 -37.137155  3.440915e-02   
16.014700   32    -37.085490  0.042510 -37.042979 -37.139442  4.400947e-02   
22.464000   33    -37.090110  0.054112 -37.035998 -37.144463  5.557541e-02   
31.510500   34    -37.098908  0.068724 -37.030184 -37.154167  6.936661e-02   
44.200100   35    -37.114818  0.086870 -37.027948 -37.171697  8.571534e-02   
62.000000   36    -37.143847  0.107941 -37.035906 -37.202837  1.043658e-01   
85.775000   37    -37.193960  0.129567 -37.064393 -37.254182  1.238925e-01   
109.550000  38    -37.253467  0.148015 -37.105453 -37.312833  1.409613e-01   
133.325000  39    -37.316385  0.169388 -37.146998 -37.373761  1.602877e-01   
157.100000  40    -37.378537  0.196570 -37.181967 -37.433853  1.843944e-01   
180.875000  41    -37.439573  0.228529 -37.211044 -37.492847  2.128703e-01   
204.650000  42    -37.499289  0.264610 -37.234679 -37.550579  2.453742e-01   
228.425000  43    -37.557545  0.304356 -37.253188 -37.606923  2.816179e-01   
252.200000  44    -37.614187  0.347391 -37.266797 -37.661769  3.213602e-01   
275.975000  45    -37.669145  0.393441 -37.275703 -37.715024  3.643766e-01   
299.750000  46    -37.722370  0.442294 -37.280076 -37.766615  4.104760e-01   
323.525000  47    -37.773830  0.493771 -37.280059 -37.816493  4.594977e-01   
347.300000  48    -37.823501  0.547721 -37.275780 -37.864603  5.112713e-01   
371.075000  49    -37.871363  0.604015 -37.2673

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')